In [9]:
client_id = '3fc718227bd34b34beb8b6aac2dbe73e'
client_secret = 'b01009306e054e6ca87406d04d2ec916'

https://github.com/codingforentrepreneurs/30-Days-of-Python/commit/9c8f45e840483133c340f10583ad6cb3f5e03fb6

https://developer.spotify.com/dashboard/login

In [10]:
import base64
import datetime
from urllib.parse import urlencode

import requests


# In[14]:


class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist' ):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)

In [11]:
spotify = SpotifyAPI(client_id, client_secret)

In [45]:
data = spotify.search("Big Life", search_type = 'track')

q=Big+Life&type=track


In [46]:
data

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Big+Life&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3zorWCDx017sz4UYP2fC9w'},
       'href': 'https://api.spotify.com/v1/artists/3zorWCDx017sz4UYP2fC9w',
       'id': '3zorWCDx017sz4UYP2fC9w',
       'name': 'Chronic Law',
       'type': 'artist',
       'uri': 'spotify:artist:3zorWCDx017sz4UYP2fC9w'}],
     'available_markets': ['AD',
      'AE',
      'AR',
      'AT',
      'AU',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HU',
      'ID',
      'IE',
      'IL',
      'IN',
      'IS',
      'IT',
      'JO',
      'JP',
      

In [6]:
tracks = ['Fernando', 'Waterloo', 'Hello']

In [49]:
for i in data['tracks']['items']:
    print(i['artists'][0]['name'])
        #print(i['album']['release_date'])
        
    

    #print(data['tracks']['items'][0]['album']['release_date'])

Chronic Law
USS (Ubiquitous Synergy Seeker)
Thomas Bergersen
Night Ranger
Freeman
Giza
HDX RECORDS
Big Sean
Big Sean
Big Time Rush
Lil Uzi Vert
Lil Uzi Vert
The Notorious B.I.G.
The Notorious B.I.G.
JAY-Z
Thomas Rhett
Thomas Rhett
Thomas Rhett
Lil Skies
The Notorious B.I.G.


In [2]:
import pandas as pd

In [3]:
df3 = pd.read_csv("songs_25.csv")

In [5]:
df3 = df3.set_index('artists')

In [6]:
series1 = df3.song

In [7]:
series1

artists
ABBA                Me And I
ABBA            My Mama Said
ABBA       Hole In Your Soul
ABBA               Cassandra
ABBA           Just A Notion
                 ...        
ZZ Top              36-22-36
ZZ Top       Deal Goin' Down
ZZ Top    Can't Stop Rockin'
ZZ Top     I Got The Message
ZZ Top          Antenna Head
Name: song, Length: 6800, dtype: object

In [40]:
artist = []
dates = []
title = []
series = pd.Series(dtype = str)
counter = 0
for k, v in series1.items():
    data = spotify.search(f"{v}", search_type = 'track')
    try:
        for i in data['tracks']['items']:
            if i['artists'][0]['name'] == f"{k}":
                tag = i['album']['release_date']
                dates.append(tag)
                artist.append(k)
                title.append(v)
        counter +=1
    except Exception:
        pass
df = pd.DataFrame({'title': title, 'date': dates, 'artist': artist})
        


q=Me+And+I&type=track
q=My+Mama+Said&type=track
q=Hole+In+Your+Soul&type=track
q=Cassandra&type=track
q=Just+A+Notion&type=track
q=Hamlet+III&type=track
q=One+Of+Us&type=track
q=The+Way+Old+Friends+Do&type=track
q=Juper+Jrouper&type=track
q=When+I+Kissed+The+Teacher&type=track
q=I%27ve+Been+Waiting+For+You&type=track
q=My+Love%2C+My+Life&type=track
q=I+Am+The+City&type=track
q=Sitting+In+The+Palmtree&type=track
q=I+Do%2C+I+Do%2C+I+Do%2C+I+Do%2C+I+Do&type=track
q=Eagle&type=track
q=Love+Has+It%27s+Ways&type=track
q=He+Is+Your+Brother&type=track
q=Man+In+The+Middle&type=track
q=Knowing+Me+Knowing+You&type=track
q=Fernando+%28In+Spanish%29&type=track
q=One+Man%2C+One+Woman&type=track
q=As+Good+As+New&type=track
q=Me+And+Bobby+And+Bobby%27s+Brother&type=track
q=Happy+Hawaii&type=track
q=Gotta+Love+It&type=track
q=St.+John&type=track
q=Last+Child&type=track
q=Once+Is+Enough&type=track
q=Mother+Popcorn&type=track
q=Pink&type=track
q=The+Farm&type=track
q=Rats+In+The+Cellar&type=track
q=Battl

q=Eye+To+Eye&type=track
q=Father&type=track
q=Rockin%27+Around+The+Christmas+Tree&type=track
q=Have+Yourself+A+Merry+Little+Christmas&type=track
q=Got+To+Let+It+Go&type=track
q=Mister+Santa&type=track
q=Sing+Your+Praise+To+The+Lord&type=track
q=Christmas+Needs+Love+To+Be+Christmas&type=track
q=Walk+Hand+In+Hand&type=track
q=An+Old+Fashioned+Love+Song&type=track
q=Promise+Me%2C+Love&type=track
q=A+Song+For+You&type=track
q=Tender+Is+The+Night&type=track
q=People&type=track
q=What+Now+My+Love&type=track
q=Say+It+Isn%27t+So&type=track
q=Show+Me&type=track
q=A+Time+For+Us&type=track
q=Almost+There&type=track
q=Unchained+Melody&type=track
q=Never+On+Sunday&type=track
q=Last+Chance&type=track
q=Wrong+For+Each+Other&type=track
q=My+Happiness&type=track
q=Quiet+Night+Of+Quiet+Stars&type=track
q=Born+Free&type=track
q=Moon+River&type=track
q=It%27s+A+Most+Unusual+Day&type=track
q=Days+Of+Wine+And+Roses&type=track
q=Still+Under+The+Weather&type=track
q=The+Very+Thought+Of+You&type=track
q=Last+T

q=You+Can+Make+Me+Free&type=track
q=Sleeping+With+The+Television+On&type=track
q=This+Night&type=track
q=Los+Angelenos&type=track
q=The+Entertainer&type=track
q=Uptown+Girl&type=track
q=A+Minor+Variation&type=track
q=I+Don%27t+Want+To+Be+Alone&type=track
q=You%27ve+Got+Me+Hummin%27&type=track
q=Scandinavian+Skies&type=track
q=Tomorrow+Is+Today&type=track
q=Tell+Her+About+It&type=track
q=I%27m+An+Old+Cowhand+%28From+The+Rio+Grande%29&type=track
q=Alexander%27s+Ragtime+Band&type=track
q=Have+You+Met+Miss+Jones%3F&type=track
q=I+Saw+Three+Ships&type=track
q=Mountain+Greenery&type=track
q=At+Last&type=track
q=For+Me+And+My+Gal+-+With+Judy+Garland&type=track
q=Pistol+Packin+Mama&type=track
q=Mexicali+Rose&type=track
q=Santa+Claus+Is+Coming+To+Town&type=track
q=I+Ain%27t+Got+Nobody&type=track
q=Here+Comes+Santa+Claus&type=track
q=Is+You+Is+Or+Is+You+Ain%27t+%28My+Baby%29%3F&type=track
q=Now+Is+The+Hour&type=track
q=Be+Careful+It%27s+My+Heart&type=track
q=On+The+Sunny+Side+Of+The+Street+%28Wi

q=Da+Doo+Ron+Ron&type=track
q=Can%27t+Smile+Without+You&type=track
q=We%27ve+Only+Just+Begun&type=track
q=Now&type=track
q=B%27wana+She+No+Home&type=track
q=A+Song+For+You&type=track
q=Sleep+Well%2C+Little+Children&type=track
q=Have+Yourself+A+Merry+Little+Christmas&type=track
q=Tryin%27+To+Get+The+Feeling+Again&type=track
q=My+Favorite+Things&type=track
q=What+Are+You+Doing+New+Year%27s+Eve%3F&type=track
q=Maybe+It%27s+You&type=track
q=Ticket+To+Ride&type=track
q=Echo&type=track
q=Love+Is+Surrender&type=track
q=Your+Baby+Doesn%27t+Love+You+Anymore&type=track
q=Winter+Wonderland&type=track
q=Silver+Bells&type=track
q=Here+Comes+My+Baby&type=track
q=Randy&type=track
q=Smash+Your+Heart&type=track
q=Love+Lives+In+The+Sky&type=track
q=Lovely+City&type=track
q=Morning+Has+Broken&type=track
q=Novim%27s+Nightmare&type=track
q=18th+Avenue&type=track
q=Sitting&type=track
q=Honeyman&type=track
q=How+Many+Times&type=track
q=I%27m+Gonna+Be+King&type=track
q=Changes+Iv&type=track
q=Mountain&type=tr

q=Driftin%27+Blues&type=track
q=Rain+Eyes&type=track
q=I+Got+A+Booking&type=track
q=I+Need+You+Baby&type=track
q=Rock+Cradle+Rock&type=track
q=My+Ding-a-ling-a-ling&type=track
q=Nadine+%28Is+It+You%3F%29&type=track
q=My+Woman&type=track
q=Let%27s+Do+Our+Thing+Together&type=track
q=Our+Little+Rendezvous&type=track
q=Ramona+Say+Yes&type=track
q=Soul+Rocking&type=track
q=I%27m+Just+A+Lucky+So+And+So&type=track
q=You+Never+Can+Tell&type=track
q=Stop+And+Listen&type=track
q=I+Love+Her+I+Love+Her&type=track
q=I+Don%27t+Take+But+A+Few+Minutes&type=track
q=Merry+Christmas+Baby&type=track
q=I+Can%27t+Believe&type=track
q=Bordeaux+In+My+Pirough&type=track
q=I%27m+Talking+About+You&type=track
q=Misery&type=track
q=Childhood+Sweetheart&type=track
q=Tulane&type=track
q=Rebel+Waltz&type=track
q=Stop+The+World&type=track
q=North+And+South&type=track
q=Train+In+Vain&type=track
q=Remote+Control&type=track
q=Fingerpoppin%27&type=track
q=Let%27s+Go+Crazy&type=track
q=Guns+On+The+Roof&type=track
q=Radio+C

...

q=Lies&type=track
q=Beautiful+Things&type=track
q=How+Do+You+Do%21&type=track
q=Blah+Blah+Blah+Blah+Blah+%28You+Broke+My+Heart%29&type=track
q=Waiting+For+The+Rain&type=track
q=Bringing+Me+Down+To+My+Knees&type=track
q=Sitting+On+The+Top+Of+The+World&type=track
q=You+Don%27t+Understand+Me&type=track
q=Call+Of+The+Wild&type=track
q=Crush+On+You&type=track
q=Things+Will+Never+Be+The+Same&type=track
q=You+Can%27t+Put+Your+Arms+Around+What%27s+Already+Gone&type=track
q=Sleeping+In+My+Car&type=track
q=One+Is+Such+A+Lonely+Number&type=track
q=Physical+Fascination&type=track
q=Give+You+Up&type=track
q=Pay+The+Price&type=track
q=The+Big+L.&type=track
q=I%27m+Under+Your+Magic+Spell&type=track
q=Opportunity+Nox&type=track
q=Big+Black+Cadillac&type=track
q=Breathe&type=track
q=So+Far+Away&type=track
q=She+Doesn%27t+Live+Here+Anymore&type=track
q=Joyride&type=track
q=Leah&type=track
q=Blue+Bayou&type=track
q=Movin%27&type=track
q=I%27m+In+A+Blue%2C+Blue+Mood&type=track
q=Girl+Like+Mine&type=track


q=If+I+Fell&type=track
q=Eleanor+Rigby&type=track
q=Day+Tripper&type=track
q=I%27ve+Got+A+Feeling&type=track
q=Hello%2C+Little+Girl&type=track
q=Lucy+In+The+Sky+With+Diamonds&type=track
q=Mr.+Moonlight&type=track
q=Anna%2C+Go+To+Him&type=track
q=Ask+Me+Why&type=track
q=Here+Comes+The+Sun&type=track
q=It+Won%27t+Be+Long&type=track
q=Don%27t+Let+Me+Down&type=track
q=If+You%27ve+Got+Trouble&type=track
q=Michelle&type=track
q=I+Want+To+Hold+Your+Hand&type=track
q=No+Reply&type=track
q=Everybody%27s+Trying+To+Be+My+Baby&type=track
q=If+I+Needed+Someone+To+Love&type=track
q=Carol&type=track
q=A+Shot+Of+Rhythm+And+Blues&type=track
q=Komm%2C+Gib+Mir+Deine+Hand&type=track
q=Ob-La-Di%2C+Ob-La-Da&type=track
q=Oh+What+A+Night&type=track
q=Don%27t+Call+On+Me&type=track
q=It%27s+Not+Too+Late&type=track
q=No+Time&type=track
q=Never+Enough&type=track
q=The+Girl+I+Left+Behind+Me&type=track
q=As+We+Go+Along&type=track
q=Bye+Bye%2C+Baby%2C+Bye+Bye&type=track
q=My+Share+Of+The+Sidewalk&type=track
q=Saturd

q=Inevitability+Of+Death&type=track
q=The+New+Maybe&type=track
q=Put+It+Off&type=track
q=I%27m+A+Werewolf+Baby&type=track
q=Frozen+In+My+Tracks&type=track
q=An+Inch+An+Hour&type=track
q=About+This+Map&type=track
q=Like+A+Song&type=track
q=Red+Light&type=track
q=Desire&type=track
q=The+Saints+Are+Coming&type=track
q=Fortunate+Son&type=track
q=Out+Of+Control&type=track
q=Ordinary+Love&type=track
q=Beat+On+The+Brat&type=track
q=God+Part+Ii&type=track
q=11+O%27clock+Tick+Tock&type=track
q=Do+You+Feel+Loved%3F&type=track
q=So+Cruel&type=track
q=Angel+Of+Harlem&type=track
q=Night+And+Day&type=track
q=Are+You+Gonna+Wait+Forever%3F&type=track
q=Babyface&type=track
q=Staring+At+The+Sun&type=track
q=Mothers+Of+The+Disappeared&type=track
q=Even+Better+Than+The+Real+Thing&type=track
q=Get+On+Your+Boots&type=track
q=The+Dream+Is+Over&type=track
q=Electrical+Storm&type=track
q=Some+Days+Are+Better+Than+Others&type=track
q=Big+Girls+Are+Best&type=track
q=Stranger+In+A+Strange+Land&type=track
q=Keep+O

q=Honky+Tonk+Blues&type=track
q=I%27m+A+Long+Way+From+Home&type=track
q=Don%27t+Cuss+The+Fiddle&type=track
q=Dreaming+My+Dreams+With+You&type=track
q=Can%27t+You+See&type=track
q=Come+With+Me&type=track
q=Good+Time+Charlie%27s+Got+The+Blues&type=track
q=It%27s+Not+Supposed+To+Be+That+Way&type=track
q=Green+River&type=track
q=Are+You+Ready+For+The+Country&type=track
q=Doesn%27t+Anybody+Know+My+Name&type=track
q=Alone&type=track
q=This+Is+The+Life&type=track
q=Phony+Calls&type=track
q=Trash+Day&type=track
q=Canadian+Idiot&type=track
q=I+Lost+On+Jeopardy&type=track
q=Fun+Zone&type=track
q=Mr.+Popeil&type=track
q=Headline+News&type=track
q=Cavity+Search&type=track
q=Melanie&type=track
q=Amish+Paradise&type=track
q=Don%27t+Download+This+Song&type=track
q=Dare+To+Be+Stupid&type=track
q=King+Of+Suede&type=track
q=Party+At+The+Leper+Colony&type=track
q=She+Never+Told+Me+She+Was+A+Mime&type=track
q=I+Love+Rocky+Road&type=track
q=That+Boy+Could+Dance&type=track
q=The+Hot+Rocks+Polka&type=track
q

q=La+Grange&type=track
q=Tube+Snake+Boogie&type=track
q=Dirty+Dog&type=track
q=2000+Blues&type=track
q=Thunderbird&type=track
q=36-22-36&type=track
q=Deal+Goin%27+Down&type=track
q=Can%27t+Stop+Rockin%27&type=track
q=I+Got+The+Message&type=track
q=Antenna+Head&type=track


In [41]:
df

,title,date,artist
0,Me And I,1980,ABBA
1,My Mama Said,1974-01-01,ABBA
2,My Mama Said,1974-03-04,ABBA
3,Hole In Your Soul,1977,ABBA
4,Hole In Your Soul,2014-01-01,ABBA
...,...,...,...
8903,Can't Stop Rockin',1985-10-28,ZZ Top
8904,Can't Stop Rockin',2019-06-14,ZZ Top
8905,Can't Stop Rockin',2003,ZZ Top
8906,I Got The Message,1985-10-28,ZZ Top


In [42]:
import pandas as pd

In [43]:
df = pd.DataFrame({'title': title, 'date': dates, 'artist': artist})

In [44]:
df.to_excel('dates.xlsx')